In [5]:
class A_Polynomial:
    ''' Abstract Polynomial Base Class'''
    def get_degree(self):
        ''' returns leading coefficient '''
        val_str = bin(self.val)
        # remove 0b prefix from string
        val_str = val_str[2:]
        for i in range(len(val_str)):
            if(val_str[i] == '1'):
                return len(val_str)-1 - i
        return 0
        
    def __str__(self):
        result = ''
        # trim leading 0b
        str_val = bin(self.val)[2:]
        for i, val in enumerate(str_val):
            cur_degree = self.get_degree()-i
            if val == '1':
                result += f'x^{cur_degree} +' 
        # trim leading plus
        result = result[0:-1]
        return result + f', bin: {bin(self.val)}, dec: {self.val}'
    


In [6]:
class IrreducablePolynomial(A_Polynomial):
    irreducable_polys = {
        4: 0b111,
        8: 0b1101,
        16: 0b11001
    }
    def __init__(self, field_num: int):
        ''' field_num - size of the field we want to use'''
        self.val = self.irreducable_polys.get(field_num, -1)
        if self.val == -1:
            raise Exception('No Binary Field of That Size Exists.')
        self.degree = None

In [7]:
from copy import deepcopy
from math import log

class Polynomial(A_Polynomial):
    def __init__(self, val: int, poly: IrreducablePolynomial, log=False):
        '''
            polynomial mod 2
            val - value of this polynomial as bit string
            poly - irreducible polynomial to mod by
        '''
        self.val = val
        self.poly = poly
        self.log = log
        self.degree = None
    
    def __add__(self, other):
        ''' xor this val with others val then mod by poly'''
        result = self.val ^ other.val
        return Polynomial(result, self.poly) % self.poly
    
    def __mul__(self, other):
        product = 0
        str_val = bin(other.val)[2:]
        for i, val in enumerate(str_val):
            if val == "1":
                shift = len(str_val)-1-i
                val_shifted = self.val << shift
                product ^= val_shifted
        #print(f'product: {bin(product)}')
        return Polynomial(product, poly=self.poly) % self.poly
    
    def pow_mod(self, base, exp, p, b):
        '''computes base**exp % p'''
        if exp==1:
            ans = base*b % p
            print(type(ans))
            return ans
        elif exp%2==0:
            return self.pow_mod(base*base % p, exp//2, p, b)
        return self.pow_mod(base*base % p, (exp-1)//2, p, base*b % p)
    
    def __pow__(self, power):
#         product = self
#         for i in range(1,power):
#             product *= self
#         return product
        return self.pow_mod(base=self, exp=power, p=self.poly, b=Polynomial(0b1, self.poly))

    def __mod__(self, other: IrreducablePolynomial):
        ''' other is an irreducable polynomial'''
        this = deepcopy(self)
        other = deepcopy(other)
        #print(f'this: {this}, irreducable: {other}')
        while(this.get_degree() >= other.get_degree()):
            #print(f'this_degree: {this.get_degree()}, irreducabledeg: {other.get_degree()}')
            diff_degree = this.get_degree() - other.get_degree()
            #print(f'this: {this}, irreducable: {other}')
            #print(f'diff_degree: {diff_degree}')
            #print(f'old_val: {this.val}')
            #print(f'{bin(this.val)} ^ {bin(other.val<<diff_degree)}')
            this.val = this.val ^ (other.val<<diff_degree)
            #print(f'new_val: {this.val}\n')
        return Polynomial(this.val, self.poly)
    def frobenius(self):
        return self**2
#     def trace(self):
#         tr(8) in F16 = 8 + 8^2 + 8^4 + 8^8
#         times = log(self.poly.val, 2)
#         times = int(times)-1
#         print(times)
#         res_sum = self
#         result = self
#         for i in range(times):
#             res_sum += result
#             result = result.frobenius()
#         return res_sum

In [8]:
# test get_leading and add
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b100101,poly=irred)
p2 = Polynomial(val=0b100,poly=irred)
print(f'p1-> degree of {p1}, {p1.get_degree()}')
print(f'p2-> degree of {p2}, {p2.get_degree()}')
print(f'p1 + p2 = {p1 + p2}')

p1-> degree of x^5 +x^2 +x^0 , bin: 0b100101, dec: 37, 5
p2-> degree of x^2 , bin: 0b100, dec: 4, 2
p1 + p2 = x^3 +x^1 , bin: 0b1010, dec: 10


In [9]:
# mod test
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1111000,poly=irred)
#print(f'p1-> degree of {p1}: {p1.get_degree()}')
#print(f'p2-> degree of {p2}: {p2.get_degree()}')
print(f'p1 % irred = {p1 % p1.poly}')

p1 % irred = x^2 +x^0 , bin: 0b101, dec: 5


In [10]:
# test multiply
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1000,poly=irred)
p2 = Polynomial(val=0b1111,poly=irred)
print(f'p1 * p2 = {p1 * p2}')

p1 * p2 = x^2 +x^0 , bin: 0b101, dec: 5


In [11]:
# test power
p1 = Polynomial(val=0b1000,poly=irred)
print(p1**2)

<class '__main__.Polynomial'>
x^3 +x^2 +x^1 +x^0 , bin: 0b1111, dec: 15


In [12]:
# discrete log trick 
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b10, poly=irred)
for i in range(2,16):
    print(f"i: {i}", f"{2}^{i}={(p1**i).val}")

<class '__main__.Polynomial'>
i: 2 2^2=4
<class '__main__.Polynomial'>
i: 3 2^3=8
<class '__main__.Polynomial'>
i: 4 2^4=9
<class '__main__.Polynomial'>
i: 5 2^5=11
<class '__main__.Polynomial'>
i: 6 2^6=15
<class '__main__.Polynomial'>
i: 7 2^7=7
<class '__main__.Polynomial'>
i: 8 2^8=14
<class '__main__.Polynomial'>
i: 9 2^9=5
<class '__main__.Polynomial'>
i: 10 2^10=10
<class '__main__.Polynomial'>
i: 11 2^11=13
<class '__main__.Polynomial'>
i: 12 2^12=3
<class '__main__.Polynomial'>
i: 13 2^13=6
<class '__main__.Polynomial'>
i: 14 2^14=12
<class '__main__.Polynomial'>
i: 15 2^15=1


In [35]:
print(p1**80)

x^3 +x^1 +x^0 , bin: 0b1011, dec: 11
